In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.documents import Document
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
from typing import Literal, List

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

print("✅ LLM initialized")

In [ ]:
# ChromaDB and Embeddings setup
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.genai import types

# Configuration
PERSIST_DIR = "../../chroma_db"
COLLECTION_NAME = "toyota_specs"
EMBED_MODEL_ID = "gemini-embedding-001"

# Initialize embeddings (uses GOOGLE_API_KEY from environment)
embeddings_model = GoogleGenerativeAIEmbeddings(
    model=EMBED_MODEL_ID,
    output_dimensionality=768
)

# Connect to vectorstore
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings_model,
    persist_directory=PERSIST_DIR
)

print(f"✅ Connected to vectorstore: {COLLECTION_NAME}")
print(f"✅ Using embedding model: {EMBED_MODEL_ID}")

In [ ]:
vectorstore.get()

In [ ]:
# Vector similarity search helper
def vector_similarity_search(
    query: str, 
    vectorstore, 
    k: int = 5
) -> List[str]:
    """Perform vector similarity search."""
    docs = vectorstore.similarity_search(query, k=k)
    return [doc.page_content for doc in docs]

In [ ]:
docs = vector_similarity_search(
    "What is the base price of the Toyota Camry?", 
    vectorstore, 
    k=5
)

In [ ]:
docs

In [ ]:
# Tool 1: Vehicle Search
@tool
def search_vehicles(query: str, max_results: int = 5) -> str:
    """
    Search Toyota vehicle database using semantic similarity.
    
    Use this tool when users need information about Toyota vehicles,
    including specifications, pricing, fuel efficiency, or comparisons.
    
    Args:
        query: Natural language search query about Toyota vehicles
        max_results: Maximum number of results to return (default: 5)
    
    Returns:
        Formatted string with vehicle information
    """
    docs = vector_similarity_search(query, vectorstore, k=max_results)
    
    result = "Vehicle Search Results:\n"
    result += "=" * 60 + "\n"
    for i, doc in enumerate(docs, 1):
        result += f"\nResult {i}:\n{doc}\n"
    result += "=" * 60
    
    return result

print("✅ search_vehicles tool defined")

In [ ]:
search_vehicles.invoke({
    "query": "Which Toyota sedan is most fuel-efficient under $30,000?",
    "max_results": 3
})

In [ ]:
# Build LangGraph workflow
tools = [search_vehicles]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """LLM node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

app = workflow.compile()
print("✅ Graph compiled")

In [ ]:
# Test: Simple vehicle search
state = {
    "messages": [
        HumanMessage(content="Which Toyota sedan is most fuel-efficient under $30,000?")
    ]
}

print("Query: Which Toyota sedan is most fuel-efficient under $30,000?")
print("=" * 70)

result = app.invoke(state)

print(f"\nTotal messages: {len(result['messages'])}")
print("\nFinal Response:")
print("=" * 70)
print(result['messages'][-1].content)
print("=" * 70)

In [ ]:
# Test: Comparison query
state2 = {
    "messages": [
        HumanMessage(content="Compare the Camry and Corolla for city driving")
    ]
}

print("Query: Compare the Camry and Corolla for city driving")
print("=" * 70)

result2 = app.invoke(state2)

print(f"\nTotal messages: {len(result2['messages'])}")
print("\nFinal Response:")
print("=" * 70)
print(result2['messages'][-1].content)
print("=" * 70)

In [ ]:
# Test: Capability query
state3 = {
    "messages": [
        HumanMessage(content="Which Toyota can tow over 5,000 lbs?")
    ]
}

print("Query: Which Toyota can tow over 5,000 lbs?")
print("=" * 70)

result3 = app.invoke(state3)

print(f"\nTotal messages: {len(result3['messages'])}")
print("\nFinal Response:")
print("=" * 70)
print(result3['messages'][-1].content)
print("=" * 70)

In [ ]:
# Test: Direct tool invocation (without LangGraph)
print("Direct Tool Test")
print("=" * 70)

direct_result = search_vehicles.invoke({
    "query": "electric Toyota with range over 200 miles",
    "max_results": 3
})

print("\nDirect Search Results:")
print("=" * 70)
print(direct_result)
print("=" * 70)

In [ ]:
# Streaming execution
state_stream = {
    "messages": [
        HumanMessage(content="Show me affordable SUVs with good fuel economy")
    ]
}

print("Streaming Execution")
print("=" * 70)
print("Query: Show me affordable SUVs with good fuel economy\n")

step_count = 0

for event in app.stream(state_stream):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: '{node_name}'")
        print("-" * 60)
        
        if "messages" in data:
            for msg in data["messages"]:
                if isinstance(msg, AIMessage):
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        print(f"  🔍 Calling {msg.tool_calls[0]['name']}")
                        print(f"     Query: {msg.tool_calls[0]['args'].get('query')}")
                    else:
                        print(f"  💬 Final response generated")
                        
                elif isinstance(msg, ToolMessage):
                    print(f"  ✅ Tool executed")
                    first_line = msg.content.split('\n')[0]
                    print(f"     Result: {first_line}")

print("\n" + "=" * 70)
print(f"Total steps: {step_count}")
print("=" * 70)